In [1]:
import numpy as np
import pandas as pd
import mnist

# data_folder  = '../../data/mnist/'
data_folder = '/Users/calvinku/Dropbox/project_dstk/dstk_data/mnist/'

In [2]:
# from glob import glob
# import os

data = pd.read_csv(data_folder + 'train.csv')

# mnist.init()
# X_train, y_train, X_test, y_test = mnist.load()

In [3]:
features, label = data.drop(['label'], axis=1), data['label']

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.3, random_state=1113)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.4, random_state=1113)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((17640, 784), (12600, 784), (17640,), (12600,))

In [ ]:
import pickle

with open('X_train_mnist.pkl', 'wb') as handle:
    pickle.dump(X_train, handle)
with open('X_test_mnist.pkl', 'wb') as handle:
    pickle.dump(X_test, handle)
with open('y_train_mnist.pkl', 'wb') as handle:
    pickle.dump(y_train, handle)
with open('y_test_mnist.pkl', 'wb') as handle:
    pickle.dump(y_test, handle)

In [ ]:
import pickle

with open('X_train_mnist.pkl', 'rb') as handle:
    X_train = pickle.load(handle)
with open('X_test_mnist.pkl', 'rb') as handle:
    X_test = pickle.load(handle)
with open('y_train_mnist.pkl', 'rb') as handle:
    y_train = pickle.load(handle)
with open('y_test_mnist.pkl', 'rb') as handle:
    y_test = pickle.load(handle)

In [66]:
from collections import namedtuple
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from collections import defaultdict

class AutoencoderKmeans(nn.Module):
    def __init__(self, input_dim, encoding_dim=10, num_clusters=10):
        super().__init__()
        # Global timestamp
        self.t = 0
        
        # Hyper parameters
        self.num_clusters = num_clusters
        self.encoding_dim = encoding_dim
        
        # Define AE architecture
#         self.fc1 = nn.Linear(input_dim, self.encoding_dim)
#         self.fc2 = nn.Linear(self.encoding_dim, input_dim)
        self.enc1 = nn.Linear(input_dim, 1000)
        self.enc2 = nn.Linear(1000, 250)
        self.enc3 = nn.Linear(250, 50)
        self.enc4 = nn.Linear(50, self.encoding_dim)
        self.dec1 = nn.Linear(self.encoding_dim, 50)
        self.dec2 = nn.Linear(50, 250)
        self.dec3 = nn.Linear(250, 1000)
        self.dec4 = nn.Linear(1000, input_dim)

        # Initialize cluster centers
        self.centers = torch.from_numpy(np.random.uniform(0, 1, (self.num_clusters, self.encoding_dim))).type(torch.FloatTensor)
        self.clusters = None
        self.AKoutput = namedtuple('AKoutput', 'k_means_loss clusters data_hidden output')
        
    def forward(self, x):
        if self.training:
#             x = self.fc1(x)

#             data_hidden = x.clone()
#             clusters = self.get_clusters(data_hidden)
#             k_means_loss = self.get_kmeans_loss(data_hidden, clusters)
#             self.centers = self.update_centers(data_hidden, clusters)

#             x = self.fc2(x)

#             output = x

#             self.t += 1

#             output_tuple = self.AKoutput(k_means_loss=k_means_loss, clusters=clusters, data_hidden=data_hidden, output=output)
#     #         return k_means_loss, clusters, data_hidden, output
#             return output_tuple
            x = self.enc1(x)
            x = F.tanh(x)
            x = self.enc2(x)
            x = F.tanh(x)
            x = self.enc3(x)
            x = F.tanh(x)
            x = self.enc4(x)
            
            data_hidden = x.clone()
            clusters = self.get_clusters(data_hidden)
            k_means_loss = self.get_kmeans_loss(data_hidden, clusters)
            self.centers = self.update_centers(data_hidden, clusters)
            
            x = self.dec1(x)
            x = F.tanh(x)
            x = self.dec2(x)
            x = F.tanh(x)
            x = self.dec3(x)
            x = F.tanh(x)
            x = self.dec4(x)
            
            output = x

            self.t += 1

            output_tuple = self.AKoutput(k_means_loss=k_means_loss, clusters=clusters, data_hidden=data_hidden, output=output)
            return output_tuple
    
        else:
#             x = self.fc1(x)
#             data_hidden = x.clone()
#             clusters = self.get_clusters(data_hidden)
#             k_means_loss = self.get_kmeans_loss(data_hidden, clusters)
#             x = self.fc2(x)
#             output = x
#             self.t += 1
#             output_tuple = self.AKoutput(k_means_loss=k_means_loss, clusters=clusters, data_hidden=data_hidden, output=output)
#             return output_tuple
            x = self.enc1(x)
            x = F.tanh(x)
            x = self.enc2(x)
            x = F.tanh(x)
            x = self.enc3(x)
            x = F.tanh(x)
            x = self.enc4(x)
            
            data_hidden = x.clone()
            clusters = self.get_clusters(data_hidden)
            k_means_loss = self.get_kmeans_loss(data_hidden, clusters)
            
            x = self.dec1(x)
            x = F.tanh(x)
            x = self.dec2(x)
            x = F.tanh(x)
            x = self.dec3(x)
            x = F.tanh(x)
            x = self.dec4(x)
        
            output = x
            self.t += 1
            output_tuple = self.AKoutput(k_means_loss=k_means_loss, clusters=clusters, data_hidden=data_hidden, output=output)
            return output_tuple
            
    def get_clusters(self, dataset):
        clusters = []
        if isinstance(self.centers, torch.autograd.Variable):
            self.centers = self.centers.data
        for data in dataset:
            clusters.append(((data - Variable(self.centers, requires_grad=False))**2).sum(1).min(0)[1][0])
        return clusters
    
    def get_kmeans_loss(self, dataset, clusters):
        loss = [0 for i in range(self.num_clusters)]
        
        for i, cluster in enumerate(clusters):
            loss[int(cluster.data)] += ((dataset[i] - Variable(self.centers[int(cluster.data)], requires_grad=False))**2).sum()
            
        return sum(loss) / len(loss)
    
    def update_centers(self, dataset, clusters):
        data_by_cluster = [[] for i in range(self.num_clusters)]
        
        for i, x in enumerate(clusters):
            data_by_cluster[int(x.data)].append(dataset[i])
        
        try:
            centers = [sum(data) / len(data) for data in data_by_cluster]
        except ZeroDivisionError:
            centers = torch.from_numpy(np.random.uniform(0, 1, (self.num_clusters, self.encoding_dim))).type(torch.FloatTensor)
            print("Cluster centers reinitialized.")
        else:
            centers = torch.stack(centers)
        
        return centers
        
    @staticmethod
    def __square_loss(v1, v2):
        loss = ((v1 - v2)**2).sum(1)[0]
        
        return loss        

In [67]:
input_dim = 784

model = AutoencoderKmeans(input_dim)

In [68]:
num_layers = len(list(model.parameters()))

for i in range(num_layers):
    print(list(model.parameters())[i].size())

torch.Size([1000, 784])
torch.Size([1000])
torch.Size([250, 1000])
torch.Size([250])
torch.Size([50, 250])
torch.Size([50])
torch.Size([10, 50])
torch.Size([10])
torch.Size([50, 10])
torch.Size([50])
torch.Size([250, 50])
torch.Size([250])
torch.Size([1000, 250])
torch.Size([1000])
torch.Size([784, 1000])
torch.Size([784])


In [69]:
criterion = nn.MSELoss()
learning_rate = 0.003
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
lambd = 0.1

In [70]:
num_epochs = 100

In [71]:
recon_losses = []
kmeans_losses = []
# kmeans_lambda = 0.001
kmeans_lambda = 0

for epoch in range(num_epochs):
    x_torch = torch.from_numpy(X_train.values).type(torch.FloatTensor)
    y_torch = torch.from_numpy(X_train.values).type(torch.FloatTensor)

    x = Variable(x_torch)
    y = Variable(y_torch)

    optimizer.zero_grad()
    
    model.train()
    output_tuple = model(x)
    cost = criterion(output_tuple.output, y) + kmeans_lambda * output_tuple.k_means_loss
    cost.backward()

    optimizer.step()
    
    recon_loss = float(((y_torch - output_tuple.output.data)**2).mean())
    kmeans_loss = float(0.001 * output_tuple.k_means_loss)
    recon_losses.append(recon_loss)
    kmeans_losses.append(kmeans_loss)
    
    if epoch % 10 == 0:
        print("Epoch {}".format(epoch))
        print("Reconstruction Loss: {0}, K-means Loss: {1}".format(recon_loss, kmeans_loss))
        
        x_torch_val = torch.from_numpy(X_val.values).type(torch.FloatTensor)
        y_torch_val = torch.from_numpy(X_val.values).type(torch.FloatTensor)
        x_val = Variable(x_torch_val)
        y_val = Variable(x_torch_val)
        model.eval()
        output_tuple_val = model(x_val)
        recon_loss_val = float(((y_torch_val - output_tuple_val.output.data)**2).mean())
        kmeans_loss_val = float(0.001 * output_tuple_val.k_means_loss)
        print("Validation: R-Loss: {0}, K-means Loss: {1}".format(recon_loss_val, kmeans_loss_val))
#     print(((y_torch - outputs.data)**2).mean())
#     print(0.00001 * k_means_loss)

Epoch 0
Reconstruction Loss: 7318.369548296176, K-means Loss: 2.6832618713378906
Validation: R-Loss: 7251.246425469867, K-means Loss: 9.43791675567627
Epoch 10
Reconstruction Loss: 5970.422578617838, K-means Loss: 58.947303771972656
Validation: R-Loss: 5805.99590705002, K-means Loss: 45.95074462890625
Epoch 20
Reconstruction Loss: 4993.699474278352, K-means Loss: 81.84081268310547
Validation: R-Loss: 4898.60415401716, K-means Loss: 60.782920837402344
Epoch 30
Reconstruction Loss: 4559.550922255236, K-means Loss: 106.4798355102539
Validation: R-Loss: 4509.717878311879, K-means Loss: 76.45944213867188
Epoch 40
Reconstruction Loss: 4425.379498230534, K-means Loss: 104.82664489746094
Validation: R-Loss: 4394.613950887665, K-means Loss: 77.61102294921875
Epoch 50
Reconstruction Loss: 4397.524989394506, K-means Loss: 113.53317260742188
Validation: R-Loss: 4371.735995865872, K-means Loss: 76.66045379638672
Epoch 60
Reconstruction Loss: 4392.056139297693, K-means Loss: 120.85234069824219
Valid

In [ ]:
# X_train_hidden, _ = model(Variable(torch.from_numpy(X_train.values).type(torch.FloatTensor)))
# X_test_hidden, _ = model(Variable(torch.from_numpy(X_test.values).type(torch.FloatTensor)))

model.eval()

output_tuple_train = model(Variable(torch.from_numpy(X_train.values).type(torch.FloatTensor)))
output_tuple_test = model(Variable(torch.from_numpy(X_test.values).type(torch.FloatTensor)))

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

# y_train.shape, len([x for x in X_train.values])
normalized_mutual_info_score(y_train, [int(x) for x in output_tuple_train.clusters])

In [ ]:
output_reshape = output_tuple_train.output.data.numpy().reshape(17640, 28, 28)

In [ ]:
output_reshape[0].shape

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(output_reshape[100], interpolation='nearest')
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train, y_train)
y_pred = lr.predict_proba(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score

print(roc_auc_score(y_test, y_pred[:,1]))

In [ ]:
X_train_hidden = X_train_hidden.data.numpy()
X_test_hidden = X_test_hidden.data.numpy()


lr = LogisticRegression()

lr.fit(X_train_hidden, y_train)
y_pred = lr.predict_proba(X_test_hidden)

In [ ]:
print(roc_auc_score(y_test, y_pred[:,1]))

In [ ]:
x = Variable(torch.ones(1), requires_grad=True)
y = 5 * (x + 1) ** 2

In [ ]:
y

In [ ]:
o = 0.5 * torch.sum(y)

In [ ]:
o

In [ ]:
o.backward()

In [ ]:
x.grad

In [ ]:
np.random.rand(100).mean()

In [ ]:
centers = torch.from_numpy(np.random.rand(10, 784)).type(torch.FloatTensor)

In [ ]:
dataset = torch.from_numpy(np.random.rand(20000, 784)).type(torch.FloatTensor)

In [ ]:
dataset.shape

In [ ]:
((dataset[0] - centers)**2).sum(1).mean()

In [ ]:
((dataset[0] - centers)**2).sum(1).max(0)

In [ ]:
[((data - centers)**2).sum(1).min(0)[1][0] for data in dataset][:10]

In [ ]:
([((data - centers)**2).sum(1) for data in dataset][:10])

In [ ]:
v1 = torch.from_numpy(np.random.rand(1, 15)).type(torch.FloatTensor)
v2 = torch.from_numpy(np.random.rand(1, 15)).type(torch.FloatTensor)

def square_loss(v1, v2):
    loss = ((v1 - v2)**2).sum(1)[0]
    
    return loss

square_loss(v1, v2)